Populate config for all the known networks

In [2]:
CHECKER_ADDR = '0xd59431D244F1B67925fB1Edd1EBa3d44348956b7'
CHECKER_ABI = '[{"inputs":[],"name":"check","outputs":[{"internalType":"bool","name":"result","type":"bool"}],"stateMutability":"view","type":"function"}]'

NETWORKS = {
    'Ethereum': 'https://ethereum.publicnode.com',
    'Binance Smart Chain': 'https://bsc.publicnode.com',
    'Optimism': 'https://optimism.publicnode.com',
    'Arbitrum One': 'https://arbitrum-one.publicnode.com',
    'Polygon': 'https://polygon-bor.publicnode.com',
    'Fantom': 'https://fantom.publicnode.com',
    'Avalanche': 'https://avalanche-c-chain.publicnode.com',
    'Gnosis Chain': 'https://gnosis.publicnode.com',
    'Aurora': 'https://mainnet.aurora.dev',
    'Base': 'https://base.publicnode.com',
}

Check all the known networks and populate `results` dictionary with data like this `{ 'Ethereum': True }`

In [3]:
from multiprocessing.pool import ThreadPool as Pool
from web3 import Web3

def is_push0_supported(network, url):
    w3 = Web3(Web3.HTTPProvider(url))
    checker = w3.eth.contract(address=CHECKER_ADDR, abi=CHECKER_ABI)
    return network, checker.functions.check().call()

with Pool(10) as p:
    results = { result[0]: result[1] for result in p.starmap(is_push0_supported, NETWORKS.items()) }

Create html table using gathered data

In [4]:
import pandas as pd

df = pd.DataFrame(
    [['Yes' if x else 'No'] for x in results.values()],
    index=pd.Index(results.keys()),
)
row_colors = ['background-color: #e6ffe6;' if x else 'background-color: #ffe6e6;' for x in results.values()]
df.style.set_table_styles([
    {'selector': 'thead', 'props': [('display', 'none')]},
    *({'selector': ' + '.join(['tr'] * (i + 1)), 'props': f'{row_colors[i]}'} for i in range(len(row_colors)))
])

,0
Ethereum,Yes
Binance Smart Chain,Yes
Optimism,Yes
Arbitrum One,Yes
Polygon,Yes
Fantom,No
Avalanche,No
Gnosis Chain,Yes
Aurora,Yes
Base,Yes
